In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch

from experiments import *
import autoencoders as ae
import wandb

In [ ]:
wandb.init(project="contrastive test (umap)")

In [ ]:
suite = ae.TrajectoryContrastiveSuite(Pendulum, contrastive_loss=ae.lifted_structured_loss(5, 1, 0.01), batch_size=512, epochs=100, do_animate=True)
model = suite.train(80, 200)

In [ ]:
loss, mse = suite.test_num(model)
print(loss, mse)

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model, 1)
print(cons.min(), cons.max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn, s=1)

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model, 2)
print(cons.min(), cons.max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn, s=1)

## UMAP & Wasserstein experiments

In [ ]:
exp = CoupledOscillator
target_embedding, dist_matrix = exp.calc_umap_wd(200, 200)

In [ ]:
target_embedding_tensor = torch.tensor(target_embedding).float()

def umap_mse(embedding, labels, lambda_=1):
    global target_embedding_tensor
    return torch.mean((embedding - target_embedding_tensor[labels.long(),])**2) * lambda_

In [ ]:
suite = ae.TrajectoryContrastiveSuite(exp, contrastive_loss=umap_mse, batch_size=256, epochs=500, do_animate=True)
model = suite.train(80, 200)

In [ ]:
loss, mse = suite.test_num(model)
print(loss, mse)

In [ ]:
x = exp.data[0]
x_prime, dyn, cons = suite.try_transform(model, x)
print(cons[:, 0].min(), cons[:, 0].max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn[:, 0], s=1)

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model, 2)
print(cons[:, 0].min(), cons[:, 0].max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn[:, 0], s=1)

### Wasserstein experiments

In [ ]:
dist_matrix_tensor = torch.tensor(dist_matrix).float()

def wasserstein_mse(embedding, labels):
    global dist_matrix_tensor
    center_loss = ae.center_loss(embedding, labels)
    dist_loss = ae.diff_between_distances(embedding, labels, dist_matrix_tensor)
    return center_loss + dist_loss

In [ ]:
suite = ae.TrajectoryContrastiveSuite(exp, batch_size=256, contrastive_loss=wasserstein_mse, epochs=1000, do_animate=True)
data = exp.contrastive_data(1)
model = suite.train(80, 200)
# model = suite.train_traj_data(data)

In [ ]:
res = suite.try_transform(model, data[:, 1:])[0]

In [ ]:
loss, mse = suite.test_num(model)
print(loss, mse)

In [ ]:
x = exp.data[0]
x_prime, dyn, cons = suite.try_transform(model, x)
print(cons[:, 0].min(), cons[:, 0].max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn[:, 0], s=1)

In [ ]:
import creating.coupled_oscillator as co

In [ ]:
x = exp.contrastive_data()[:, 1:]
x_prime, dyn, cons = suite.try_transform(model, x)
print(cons[:, 0].min(), cons[:, 0].max())
cons0, cons1 = cons[:, 0], cons[:, 1]
y_true = [co.energy2(e) for e in x]
plt.scatter(cons0, cons1, c=y_true, s=0.01)
plt.scatter(cons0[:1000], cons1[:1000], c=y_true[:1000], s=10)